In [438]:
import tarfile
import functools
import numpy as np
import spacy
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import itertools
from collections import Counter
from glob import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
nlp = spacy.load('en')

In [372]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    doc = nlp(sent)
    #return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]
    return [sent.string.strip() for sent in doc], [sent.pos_ for sent in doc]

In [373]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [374]:
def get_stories(f, only_supporting=False, max_length=None):
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) 
            for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [498]:
#Now vectorize_stories() will output xqpos and xpos
def vectorize_stories(data, word_idx, pos_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    xspos = []
    xqspos = []
    for story, query, answer in data:
        x = [word_idx[w] for sent in story[::2] for w in sent ]
        xpos = [pos_idx[w] for sent in story[1::2] for w in sent ]
        xq = [word_idx[w] for w in query[0]]
        xqpos = [pos_idx[w] for w in query[1]]
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(xpos)
        xqs.append(xq)
        xqspos.append(xqpos)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen),pad_sequences(xspos, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), pad_sequences(xqspos, maxlen=query_maxlen), np.array(ys)

In [486]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [487]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [503]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

In [504]:
#Changed vocab so that it only takes the sentences not the POS
#Also added POS set
vocab = set()
pos = set()
for story, q, answer in train + test:
    vocab |= set(story[0] + q[0] + [answer])
    pos |= set(story[1] + q[1])
vocab = sorted(vocab)
pos = sorted(pos)

In [505]:
#Added pos_idx and pos_size
vocab_size = len(vocab) + 1
pos_size = len(pos) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
pos_idx = dict((c, i + 1) for i, c in enumerate(pos))
story_maxlen = max(map(len, (x for x, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, x, _ in train + test)))

In [506]:
#Added corresponding variables to the new vectorize_stories() output
x, xpos, xq, xqpos, y = vectorize_stories(train, word_idx, pos_idx, story_maxlen, query_maxlen)
tx, txpos, txq, txqpos, ty = vectorize_stories(test, word_idx,pos_idx, story_maxlen, query_maxlen)

In [507]:
#Added corresponding prints of the shapes of xqpos and xpos for reference
print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xpos.shape = {}'.format(xpos.shape))
print('xq.shape = {}'.format(xq.shape))
print('xqpos.shape = {}'.format(xqpos.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']
x.shape = (1000, 20)
xpos.shape = (1000, 20)
xq.shape = (1000, 2)
xqpos.shape = (1000, 2)
y.shape = (1000, 22)
story_maxlen, query_maxlen = 20, 2


In [509]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') 
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
encoded_sentence = layers.Dropout(0.3)(encoded_sentence)

question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
encoded_question = layers.Dropout(0.3)(encoded_question)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(encoded_question)
encoded_question = layers.RepeatVector(story_maxlen)(encoded_question)

pos = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_pos = layers.Embedding(pos_size, EMBED_HIDDEN_SIZE)(pos)
encoded_pos = layers.Dropout(0.3)(encoded_pos)

qpos = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_qpos = layers.Embedding(pos_size, EMBED_HIDDEN_SIZE)(qpos)
encoded_qpos = layers.Dropout(0.3)(encoded_qpos)
encoded_qpos = RNN(EMBED_HIDDEN_SIZE)(encoded_qpos)
encoded_qpos = layers.RepeatVector(story_maxlen)(encoded_qpos)

merged = layers.add([encoded_sentence, encoded_pos, encoded_question, encoded_qpos])
merged = RNN(SENT_HIDDEN_SIZE)(merged)
merged = layers.Dropout(0.3)(merged)
preds = layers.Dense(vocab_size, activation='softmax')(merged)

model = Model([sentence,pos,question, qpos], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [510]:
print('Training')
model.fit([x,xpos,xq,xqpos], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx,txpos,txq,txqpos], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 6s - loss: 2.3772 - acc: 0.1474 - val_loss: 1.8474 - val_acc: 0.0800